## Homework 4
*For any questions or concerns please email your instructor at `smanna@scu.edu`*
### Due - 06/01/2018 - 11:59p - 100 points

**Name:** `<Alma Niu>`

**Email** `<aniu@scu.edu>`

**Objectives:** 
* Make you conversant with word embedding using word2vec
* Familiarity with `gensim`

**Problem Statement:** 
You are given some amazon reviews on `Musical Instruments`. Your task is to clean your data, separate out sentences, and then use them to train your `word2vec` model. After your model is trained, then you have to compute the following:
* print top 10 words based on term frequency (make sure they are not stop-words)
* print `most similar` top 5 words for each of the above words
* randomly pick 10 words, and print their `similarity` in the format: `similarity between a and b is ____`. If $sim(a,b) = sim(b,a)$, then you should print only one.

**Submission Instructions:** 
* Please download `hw4.ipynb` and `reviews_Musical_Instruments_5.json` and insert cells in the `hw4.ipynb` to complete your homework and submit the same file to Camino under `Homework` $\rightarrow$ `hw4`.
* Please make sure you type your `Name` and `Email` on top of your submission file in the placeholder above. 
* Feel free to use your in-class examples `week7-nltk.ipynb` and `week8-word2Vec.ipynb` to work on this homework.

**Dataset:** The original [data](http://jmcauley.ucsd.edu/data/amazon/) contains reviews from amazon on `Musical Instruments`. 

**Honor Code:** You are expected to complete the homework on your own. Solutions might exist elsewhere, but you are not allowed to copy them. 

In [1]:
# import necessary libraries
# TODO
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import json 
import csv

In [2]:

# load the data into pandas dataframe
# make sure the data is in json format
# TODO
url='reviews_Musical_Instruments_5.json'
#df=pd.read_json(url,orient='columns')

with open(url) as json_file:
    data=json_file.readlines()
    data=list(map(json.loads,data))
df=pd.DataFrame(data)

print('Load Data')
df.head()


Load Data


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5.0,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5.0,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5.0,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5.0,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5.0,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [3]:
# View information about the data
df.info

# TODO

<bound method DataFrame.info of              asin   helpful  overall  \
0      1384719342    [0, 0]      5.0   
1      1384719342  [13, 14]      5.0   
2      1384719342    [1, 1]      5.0   
3      1384719342    [0, 0]      5.0   
4      1384719342    [0, 0]      5.0   
5      B00004Y2UT    [0, 0]      5.0   
6      B00004Y2UT    [0, 0]      5.0   
7      B00004Y2UT    [0, 0]      3.0   
8      B00004Y2UT    [0, 0]      5.0   
9      B00004Y2UT    [0, 0]      5.0   
10     B00004Y2UT    [6, 6]      5.0   
11     B00005ML71    [0, 0]      4.0   
12     B00005ML71    [0, 0]      3.0   
13     B00005ML71    [0, 0]      5.0   
14     B00005ML71    [0, 0]      5.0   
15     B00005ML71    [0, 0]      2.0   
16     B000068NSX    [0, 0]      4.0   
17     B000068NSX    [0, 0]      5.0   
18     B000068NSX    [3, 3]      5.0   
19     B000068NSX    [0, 0]      5.0   
20     B000068NSX    [0, 0]      4.0   
21     B000068NSX    [0, 0]      5.0   
22     B000068NSX    [0, 0]      4.0   
23     B

In [4]:
# Convert all the review text into a long string and print its length
longString=""
for i in range(len(df.reviewText)):
    longString+=df.reviewText[i]
print(len(longString)) 

4986118


In [5]:
# Load the tokenizer
# tokenize into sentences
# TOD0
import nltk

sentences=[i for i in nltk.sent_tokenize(longString)] #suppose to break down to array sentence
print (sentences[0]) #test

Not much to write about here, but it does exactly what it's supposed to.


In [6]:
# clean and split sentences into words
# write a function to do this
# TODO

"""Split into words and remove stopwords"""
from nltk.corpus import stopwords
def clean_split(x):
    x=nltk.word_tokenize(x) #array of words
    stop_words=set(stopwords.words("english"))
    filtered_x=[w for w in x if not w in stop_words and w!=',' and w!='.' and w!="n't" and w!='(' and w!=')'
               and w!='!' and w!='It' and w!="'s" and w!='I' and w!='The']
    return filtered_x
"""
x=clean_split(longString) #array of words
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
filtered_x=[w for w in x if not w in stop_words]
"""
words=clean_split(longString)
print(words[0:11]) #print first 11 words as test to see output array

[u'Not', u'much', u'write', u'exactly', u'supposed', u'filters', u'pop', u'sounds', u'recordings', u'much', u'crisp']


In [97]:
# might want to compute the word and their frequency and store somewhere
# you will need it later in the code
# TODO

freq=nltk.FreqDist(words) # vocab=unique words, tokens=words

print(freq['write']) #example that shows frequency of the word 'write' appearing
"""
freq.plot(20,cumulative=False)
for key, val in freq.items():
    #print(str(key) + ':' + str(val))
"""
print(freq)


42
<FreqDist with 40648 samples and 506175 outcomes>


In [98]:
# clean each raw sentences and build the list of sentences
# ideally you are calling the funtion on each sentence you tokenized earlier
# TODO
"""Already done above"""

setup=[]
count=0
for i in sentences:
    setup.append(clean_split(i))
#print(setup[0:5]) #print first 5 sentences with each word
type(setup)

list

In [99]:
# check a sentence by printing it
# TODO
print(setup[0])
print(setup[1])

[u'Not', u'much', u'write', u'exactly', u'supposed']
[u'filters', u'pop', u'sounds']


In [100]:
# count number of tokens in the dataset
# TODO
from collections import Counter
c=Counter(words)
len(words) #number of tokens, after removing stopwords

506175

In [10]:
# do the multiprocessing (optional)
# to speed up, you might want to look for this option in the documentation
# my code does not have it
"""Optional"""

In [101]:
# Build you word2vec model
# TODO

model = Word2Vec(setup, min_count=1)

In [102]:
# summarize the loaded model -- simply print your model
# TODO
print(model)


Word2Vec(vocab=40636, size=100, alpha=0.025)


In [103]:
# print corpus_count
# TODO
from collections import Counter
c=Counter(words)
c['sounds'] # show count of words

996

In [93]:
# Retrieve all of the vectors from a trained model as follows:
# TODO
X = model[model.wv.vocab]
print(X)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[[-3.02206837e-02 -2.46135797e-02 -1.23986844e-02 ...  1.22684194e-02
  -1.64362565e-02 -2.65085567e-02]
 [-7.03531206e-02 -8.48237425e-02 -5.31118959e-02 ...  4.86954004e-02
  -2.22093239e-02 -6.59034774e-02]
 [-2.48812005e-01 -2.13216916e-01 -3.31481814e-01 ...  5.81422299e-02
  -2.57652700e-01 -2.96832025e-01]
 ...
 [-6.54190197e-04 -1.02244476e-02  4.63236030e-03 ...  4.59633535e-03
   3.02415015e-03 -1.21388475e-04]
 [-1.18139293e-02 -9.27053741e-04 -1.78084299e-02 ...  3.50240269e-03
  -1.55229121e-02 -1.73608866e-02]
 [-1.82788372e-02 -6.33648597e-03 -1.08070029e-02 ...  8.75958521e-03
  -1.71680525e-02 -1.67488903e-02]]


In [104]:
# print top 10 words based on term frequency (make sure they are not stop-words)
# TODO

freq.most_common(10)

[(u'guitar', 5078),
 (u'one', 4040),
 (u'like', 3766),
 (u'use', 3593),
 (u'great', 3389),
 (u'sound', 3374),
 (u'good', 3302),
 (u'strings', 3165),
 (u'well', 2696),
 (u'get', 2576)]

In [112]:
# print most similar top 5 words for each of the above words
# TODO
top_ten=['guitar','one','like','use','great','sound','good','strings','well','get']
for i in top_ten:
    print(model.most_similar(positive=i,topn=5))
    print ('\n')

[(u'electric', 0.9554224014282227), (u'acoustic', 0.95229572057724), (u'classical', 0.9422458410263062), (u'hollow-body', 0.9362406730651855), (u'player', 0.9274585247039795)]


[(u'bag.I', 0.9673309326171875), (u'one.I', 0.9621098637580872), (u'stand.I', 0.9616100788116455), (u'first', 0.9613887071609497), (u'another', 0.9594484567642212)]


[(u'terrific', 0.9445079565048218), (u'awesome', 0.9412147402763367), (u'amazing', 0.9401862025260925), (u'fantastic', 0.934888482093811), (u'cool', 0.9340112209320068)]


[(u'put', 0.9571376442909241), (u'keep', 0.942513108253479), (u'gigs', 0.9423750638961792), (u'place', 0.9393409490585327), (u'stage', 0.9353340268135071)]


[(u'good', 0.9476885199546814), (u'wonders', 0.9254891872406006), (u'fine', 0.9190893173217773), (u'awesome', 0.9162129163742065), (u'charm', 0.9141053557395935)]


[(u'sounds', 0.9612590074539185), (u'tone', 0.9480122923851013), (u'sounding', 0.9272791743278503), (u'gritty', 0.9082261919975281), (u'feel', 0.904307365417480

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


In [148]:
# randomly pick 10 words, and print their similarity (as described in your problem statement)
random_words=['dogs','run','engineering','music','states','guitar','running','math','song','map']

"""I made my own algorithm to compare all 10 words with each other"""
while len(random_words)>1:
    for i in range(1,len(random_words)):
        print("similarity between "+ random_words[0]+ " and " +random_words[i]+ " is ")
        print(model.similarity(random_words[0],random_words[i]))
    print('\n')
    random_words.pop(0)


similarity between dogs and run is 
0.9752997946437195
similarity between dogs and engineering is 
0.9663126931574223
similarity between dogs and music is 
0.8835029317074751
similarity between dogs and states is 
0.9608456564103068
similarity between dogs and guitar is 
0.7489931941505424
similarity between dogs and running is 
0.9884063556628747
similarity between dogs and math is 
0.1593032678415771
similarity between dogs and song is 
0.9743961560722639
similarity between dogs and map is 
0.9815566198636134


similarity between run and engineering is 
0.9584802051150003
similarity between run and music is 
0.8700171974021835
similarity between run and states is 
0.9424476003585714
similarity between run and guitar is 
0.6760652104394455
similarity between run and running is 
0.9732412049585105
similarity between run and math is 
0.21920930902581176
similarity between run and song is 
0.9473708645108365
similarity between run and map is 
0.9565857775661402


similarity between engin

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
